In [ ]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex客户端库：使用自定义容器为在线预测训练表格回归模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_online_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/master/notebooks/community/gapic/custom/showcase_custom_tabular_regression_online_container.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex客户端库来训练使用自定义容器，并部署一个用于在线预测的自定义表格回归模型。

### 数据集

本教程使用的数据集是[波士顿房价数据集](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html)。您在本教程中使用的数据集版本已内置于TensorFlow中。经过训练的模型预测房屋的中位价格，单位为1千美元。

### 目标

在本教程中，您将使用Vertex客户端库从自定义Docker容器中的Python脚本创建自定义模型，然后通过发送数据对部署的模型进行预测。您也可以使用`gcloud`命令行工具或在Google Cloud Console上在线创建自定义模型。

执行的步骤包括：

- 创建一个用于训练模型的Vertex自定义作业。
- 使用自定义容器训练TensorFlow模型。
- 检索并加载模型工件。
- 查看模型评估。
- 上传模型作为Vertex `Model`资源。
- 将`Model`资源部署到服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`资源。

### 成本

本教程使用 Google Cloud（GCP）的计费组件：

* Vertex AI
* 云存储

了解 [Vertex AI
定价](https://cloud.google.com/vertex-ai/pricing) 和 [云存储
定价](https://cloud.google.com/storage/pricing)，并使用 [定价
计算器](https://cloud.google.com/products/calculator/)
根据您预计的使用情况生成成本估算。

## 安装

安装最新版本的Vertex客户端库。

In [ ]:
import os
import sys

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

! pip3 install -U google-cloud-aiplatform $USER_FLAG

安装最新的google-cloud-storage库的GA版本。

In [ ]:
! pip3 install -U google-cloud-storage $USER_FLAG

### 重新启动核心

当您安装了Vertex客户端库和Google *cloud-storage*后，您需要重新启动笔记本核心，以便它能够找到这些软件包。

In [ ]:
if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 开始之前

### GPU 运行时

*如果有这个选项，请确认在 GPU 运行时下运行这个笔记本。在 Colab 中，选择* **运行时 > 修改运行时类型 > GPU**

### 设置您的 Google Cloud 项目

**无论您的笔记本环境如何，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建帐户时，您将获得300美元的免费信用额度用于您的计算/存储成本。

2. [确保您的项目已启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用 Vertex API 和 Compute Engine API。](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) 已经安装在 Google Cloud Notebook 中。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格以确保
Cloud SDK 在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter 将以 `!` 为前缀的行作为 shell 命令运行，并将以 `$` 为前缀的 Python 变量插入这些命令中。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于本笔记本的其余操作。以下是Vertex支持的地区。我们建议您选择离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您可能无法使用多区域存储桶来训练Vertex。并非所有地区都支持所有Vertex服务。有关各地区的最新支持情况，请参阅 [Vertex 位置文档](https://cloud.google.com/vertex-ai/docs/general/locations)

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

时间戳

如果您正在进行实时教程会话，您可能正在使用共享测试帐户或项目。为了避免用户之间在创建的资源上发生名称冲突，您需要为每个实例会话创建一个时间戳，并附加到将在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### 验证您的Google Cloud帐户

**如果您正在使用Google Cloud Notebook**，您的环境已经经过身份验证。跳过这一步。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证，通过oAuth验证您的帐户。

**否则**，请按照以下步骤操作：

在Cloud Console中，转到[创建服务账户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**点击创建服务账户**。

在**服务账户名称**字段中输入一个名称，然后点击**创建**。

在**将此服务账户授予项目访问权限**部分，点击角色下拉列表。在过滤框中输入"Vertex"，选择**Vertex管理员**。在过滤框中输入"存储对象管理员"，选择**存储对象管理员**。

点击创建。包含您密钥的JSON文件将下载到您的本地环境。

在下面的单元格中将您的服务账户密钥路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，并运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用哪种笔记本环境，都需要完成以下步骤。**

当您使用Vertex客户端库提交自定义训练作业时，您需要将包含训练代码的Python软件包上传到一个云存储桶中。Vertex会从这个软件包中运行代码。在本教程中，Vertex还会将您的作业产生的训练模型保存在同一个存储桶中。然后您可以根据这个输出创建一个`Endpoint`资源，以便用于提供在线预测。

请在下面设置您的云存储桶的名称。存储桶的名称必须在所有Google Cloud项目中是全局唯一的，包括您组织外的项目。

In [ ]:
BUCKET_NAME = "gs://[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "gs://[your-bucket-name]":
    BUCKET_NAME = "gs://" + PROJECT_ID + "aip-" + TIMESTAMP

只有在您的存储桶不存在时才能运行以下单元格来创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_NAME

最后，通过检查其内容来验证对您的云存储桶的访问权限。

In [ ]:
! gsutil ls -al $BUCKET_NAME

### 设置变量

接下来，设置一些在教程中使用的变量。
### 导入库并定义常量

导入Vertex客户端库

将Vertex客户端库导入到我们的Python环境中。

In [ ]:
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### 顶点常量

为Vertex设置以下常量：

- `API_ENDPOINT`：顶点API服务的数据集、模型、作业、管道和终端点服务终点。
- `PARENT`：顶点数据集、模型、作业、管道和终端资源的位置根路径。

In [ ]:
# API service endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

####硬件加速器

设置硬件加速器（例如GPU）用于训练和预测。

设置变量`TRAIN_GPU/TRAIN_NGPU`和`DEPLOY_GPU/DEPLOY_NGPU`，以使用支持GPU的容器映像以及分配给虚拟机（VM）实例的GPU数量。例如，要在每个VM上分配4个Nvidia Telsa K80 GPU并使用GPU容器映像，您可以指定：

（aip.AcceleratorType.NVIDIA_TESLA_K80，4）

对于GPU，可用的加速器包括：
  - aip.AcceleratorType.NVIDIA_TESLA_K80
  - aip.AcceleratorType.NVIDIA_TESLA_P100
  - aip.AcceleratorType.NVIDIA_TESLA_P4
  - aip.AcceleratorType.NVIDIA_TESLA_T4
  - aip.AcceleratorType.NVIDIA_TESLA_V100

否则，指定`(None, None)`以在CPU上运行容器映像。

*注意*：在TF 2.3之前发布的TF版本对GPU支持将无法加载本教程中的自定义模型。这是一个已知问题，在TF 2.3中已修复 -- 产生在serving功能中生成的静态图操作。如果在自己的自定义模型上遇到此问题，请使用支持GPU的TF 2.3容器映像。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_GPU"):
    TRAIN_GPU, TRAIN_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_TRAIN_GPU")),
    )
else:
    TRAIN_GPU, TRAIN_NGPU = (aip.AcceleratorType.NVIDIA_TESLA_K80, 1)

if os.getenv("IS_TESTING_DEPOLY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aip.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPOLY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

####容器（Docker）镜像

接下来，我们将设置用于预测的Docker容器镜像

- 将变量`TF`设置为容器镜像的TensorFlow版本。例如，`2-1`表示版本2.1，`1-15`表示版本1.15。以下列表显示了一些可用的预构建镜像:

  - TensorFlow 1.15
    - `gcr.io/cloud-aiplatform/prediction/tf-cpu.1-15:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf-gpu.1-15:latest`
  - TensorFlow 2.1
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-1:latest`
  - TensorFlow 2.2
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-2:latest`
  - TensorFlow 2.3
    - `gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-3:latest`
    - `gcr.io/cloud-aiplatform/prediction/tf2-gpu.2-3:latest`
  - XGBoost
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-2:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.1-1:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-90:latest`
    - `gcr.io/cloud-aiplatform/prediction/xgboost-cpu.0-82:latest`
  - Scikit-learn
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-23:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-22:latest`
    - `gcr.io/cloud-aiplatform/prediction/sklearn-cpu.0-20:latest`

要获取最新列表，请参阅[用于预测的预构建容器](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers)

In [ ]:
if os.getenv("IS_TESTING_TF"):
    TF = os.getenv("IS_TESTING_TF")
else:
    TF = "2-1"

if TF[0] == "2":
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf2-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf2-cpu.{}".format(TF)
else:
    if DEPLOY_GPU:
        DEPLOY_VERSION = "tf-gpu.{}".format(TF)
    else:
        DEPLOY_VERSION = "tf-cpu.{}".format(TF)

DEPLOY_IMAGE = "gcr.io/cloud-aiplatform/prediction/{}:latest".format(DEPLOY_VERSION)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU)

机器类型

接下来，设置用于训练和预测的机器类型。

- 设置变量`TRAIN_COMPUTE`和`DEPLOY_COMPUTE`来配置用于训练和预测的VM的计算资源。
 - `机器类型`
     - `n1-standard`: 每个vCPU 3.75GB内存。
     - `n1-highmem`: 每个vCPU 6.5GB内存
     - `n1-highcpu`: 每个vCPU 0.9 GB内存
 - `vCPUs`: 数量为\[2, 4, 8, 16, 32, 64, 96 \]

*注意：以下不支持训练：*

 - `standard`: 2个vCPUs
 - `highcpu`: 2、4和8个vCPUs

*注意：您也可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU*。

In [ ]:
if os.getenv("IS_TESTING_TRAIN_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_TRAIN_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
TRAIN_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", TRAIN_COMPUTE)

if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Deploy machine type", DEPLOY_COMPUTE)

# 教程

现在，您已经准备好开始创建自己的自定义模型，并为波士顿住房进行训练。

设置客户端

Vertex客户端库采用客户端/服务器模型。 在您的端（Python脚本）中，您将创建一个客户端，该客户端向Vertex服务器发送请求并接收响应。

在本教程中的不同步骤中，您将使用不同的客户端。 因此，请提前设置它们全部。

- 用于`Model`资源的模型服务。
- 用于部署的终端服务。
- 用于批处理作业和自定义训练的作业服务。
- 用于提供服务的预测服务。

In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


clients = {}
clients["job"] = create_job_client()
clients["model"] = create_model_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()

for client in clients.items():
    print(client)

训练模型

使用容器镜像有两种方式可以训练一个自定义模型：

- **使用Google Cloud预构建的容器**。如果使用预构建的容器，您还需要指定一个要安装到容器镜像中的Python包。这个Python包包含了用于训练自定义模型的代码。

- **使用您自己的自定义容器镜像**。如果使用自己的容器，容器需要包含用于训练自定义模型的代码。

### 创建一个Docker文件

在本教程中，您将使用自己的自定义容器来训练一个波士顿房屋模型。

要使用自己的自定义容器，您需要创建一个Docker文件。首先，您需要为容器组件创建一个目录。

### 检查培训包

#### 包布局

在开始培训之前，您将查看如何为定制培训工作组装Python包。当解压缩时，该包包含以下的目录/文件布局。

- PKG-INFO
- README.md
- setup.cfg
- setup.py
- trainer
  - \_\_init\_\_.py
  - task.py

文件`setup.cfg`和`setup.py`是安装包到Docker镜像操作环境的说明。

文件`trainer/task.py`是执行定制培训工作的Python脚本。*注意*，当我们在工作组规范中提到它时，我们将目录斜杠替换为点号(`trainer.task`)并丢弃文件后缀(`.py`)。

#### 包装配

在以下单元格中，您将组装培训包。

In [ ]:
# Make folder for Python training script
! rm -rf custom
! mkdir custom

# Add package information
! touch custom/README.md

setup_cfg = "[egg_info]\n\ntag_build =\n\ntag_date = 0"
! echo "$setup_cfg" > custom/setup.cfg

setup_py = "import setuptools\n\nsetuptools.setup(\n\n    install_requires=[\n\n        'tensorflow_datasets==1.3.0',\n\n    ],\n\n    packages=setuptools.find_packages())"
! echo "$setup_py" > custom/setup.py

pkg_info = "Metadata-Version: 1.0\n\nName: Boston Housing tabular regression\n\nVersion: 0.0.0\n\nSummary: Demostration training script\n\nHome-page: www.google.com\n\nAuthor: Google\n\nAuthor-email: aferlitsch@google.com\n\nLicense: Public\n\nDescription: Demo\n\nPlatform: Vertex"
! echo "$pkg_info" > custom/PKG-INFO

# Make the training subfolder
! mkdir custom/trainer
! touch custom/trainer/__init__.py

#### Task.py 内容

在下一个单元格中，您将编写训练脚本 task.py 的内容。我不会详细说明，只是让您浏览一下。概括地说：

- 从命令行获取保存模型工件的目录（`--model_dir`），如果未指定，则从环境变量 `AIP_MODEL_DIR` 中获取。
- 从 TF.Keras 内置数据集中加载波士顿房屋数据集。
- 使用 TF.Keras 模型 API 构建一个简单的深度神经网络模型。
- 编译模型（`compile()`）。
- 根据参数 `args.distribute` 设置训练分发策略。
- 使用 `args.epochs` 指定的 epochs 训练模型（`fit()`）。
- 将训练后的模型保存到指定的模型目录中（`save(args.model_dir)`）。
- 将每个特征的最大值保存到指定的参数文件中 `f.write(str(params))`。

In [ ]:
%%writefile custom/trainer/task.py
# Single, Mirror and Multi-Machine Distributed Training for Boston Housing

import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow.python.client import device_lib
import numpy as np
import argparse
import os
import sys
tfds.disable_progress_bar()

parser = argparse.ArgumentParser()
parser.add_argument('--model-dir', dest='model_dir',
                    default=os.getenv('AIP_MODEL_DIR'), type=str, help='Model dir.')
parser.add_argument('--lr', dest='lr',
                    default=0.001, type=float,
                    help='Learning rate.')
parser.add_argument('--epochs', dest='epochs',
                    default=20, type=int,
                    help='Number of epochs.')
parser.add_argument('--steps', dest='steps',
                    default=100, type=int,
                    help='Number of steps per epoch.')
parser.add_argument('--distribute', dest='distribute', type=str, default='single',
                    help='distributed training strategy')
parser.add_argument('--param-file', dest='param_file',
                    default='/tmp/param.txt', type=str,
                    help='Output file for parameters')
args = parser.parse_args()

print('Python Version = {}'.format(sys.version))
print('TensorFlow Version = {}'.format(tf.__version__))
print('TF_CONFIG = {}'.format(os.environ.get('TF_CONFIG', 'Not found')))

# Single Machine, single compute device
if args.distribute == 'single':
    if tf.test.is_gpu_available():
        strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
    else:
        strategy = tf.distribute.OneDeviceStrategy(device="/cpu:0")
# Single Machine, multiple compute device
elif args.distribute == 'mirror':
    strategy = tf.distribute.MirroredStrategy()
# Multiple Machine, multiple compute device
elif args.distribute == 'multi':
    strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

# Multi-worker configuration
print('num_replicas_in_sync = {}'.format(strategy.num_replicas_in_sync))


def make_dataset():

  # Scaling Boston Housing data features
  def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float)
    return feature, max

  (x_train, y_train), (x_test, y_test) = tf.keras.datasets.boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
  )
  params = []
  for _ in range(13):
    x_train[_], max = scale(x_train[_])
    x_test[_], _ = scale(x_test[_])
    params.append(max)

  # store the normalization (max) value for each feature
  with tf.io.gfile.GFile(args.param_file, 'w') as f:
    f.write(str(params))
  return (x_train, y_train), (x_test, y_test)


# Build the Keras model
def build_and_compile_dnn_model():
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(128, activation='relu', input_shape=(13,)),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(1, activation='linear')
  ])
  model.compile(
      loss='mse',
      optimizer=tf.keras.optimizers.RMSprop(learning_rate=args.lr))
  return model

NUM_WORKERS = strategy.num_replicas_in_sync
# Here the batch size scales up by number of workers since
# `tf.data.Dataset.batch` expects the global batch size.
BATCH_SIZE = 16
GLOBAL_BATCH_SIZE = BATCH_SIZE * NUM_WORKERS

with strategy.scope():
  # Creation of dataset, and model building/compiling need to be within
  # `strategy.scope()`.
  model = build_and_compile_dnn_model()

# Train the model
(x_train, y_train), (x_test, y_test) = make_dataset()
model.fit(x_train, y_train, epochs=args.epochs, batch_size=GLOBAL_BATCH_SIZE)
model.save(args.model_dir)

#### 写入Docker文件内容

将您的代码容器化的第一步是创建一个Docker文件。在您的Docker文件中，您将包括运行容器镜像所需的所有命令。它将安装您正在使用的所有库，并设置用于训练代码的入口点。

1. 从TensorFlow存储库中安装预定义的深度学习镜像容器。
2. 复制Python训练代码，稍后将显示。
3. 将入口点设置为Python训练脚本为`trainer/task.py`。请注意，在ENTRYPOINT命令中省略了`.py`，因为它是暗示的。

In [ ]:
%%writefile custom/Dockerfile

FROM gcr.io/deeplearning-platform-release/tf2-cpu.2-1
WORKDIR /root

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.task"]

#### 在本地构建容器

接下来，您将为您的客户容器提供一个名称，您将在将其提交到Google容器注册表时使用该名称。

In [ ]:
TRAIN_IMAGE = "gcr.io/" + PROJECT_ID + "/boston:v1"

接下来，建造容器。

In [ ]:
! docker build custom -t $TRAIN_IMAGE

测试本地容器

在您的笔记本实例中运行容器，以确保它能正常工作。您将运行5个迭代。

In [ ]:
! docker run $TRAIN_IMAGE --epochs=5

#### 注册自定义容器

当您在本地运行容器完成后，推送到Google容器注册表。

In [ ]:
! docker push $TRAIN_IMAGE

将培训脚本存储在您的云存储桶中

接下来，您将培训文件夹打包成一个压缩的tar球，然后存储在您的云存储桶中。

In [ ]:
! rm -f custom.tar custom.tar.gz
! tar cvf custom.tar custom
! gzip custom.tar
! gsutil cp custom.tar.gz $BUCKET_NAME/trainer_boston.tar.gz

## 准备您的定制职位规格

现在您的客户已经准备好了，您的第一步是为您的定制培训工作创建一个职位规格。这份职位规格将包括以下内容：

- `worker_pool_spec`：用于培训的机器类型规格以及数量（单个或分布式）
- `container_spec`：定制容器的规格。

### 准备您的机器规格

现在为您的自定义训练作业定义机器规格。这会告诉Vertex应该为训练提供什么类型的机器实例。
- `machine_type`：要提供的GCP实例的类型，例如n1-standard-8。
- `accelerator_type`：（如果有的话）硬件加速器的类型。在本教程中，如果您之前设置了变量`TRAIN_GPU != None`，则您正在使用GPU；否则您将使用CPU。
- `accelerator_count`：加速器的数量。

In [ ]:
if TRAIN_GPU:
    machine_spec = {
        "machine_type": TRAIN_COMPUTE,
        "accelerator_type": TRAIN_GPU,
        "accelerator_count": TRAIN_NGPU,
    }
else:
    machine_spec = {"machine_type": TRAIN_COMPUTE, "accelerator_count": 0}

### 准备您的磁盘规格

（可选）现在定义自定义训练作业的磁盘规格。这将告诉 Vertex 在每个机器实例中为训练提供什么类型和大小的磁盘。

- `boot_disk_type`：SSD 或标准。SSD 更快，标准更便宜。默认为 SSD。
- `boot_disk_size_gb`：磁盘的大小，单位为 GB。

In [ ]:
DISK_TYPE = "pd-ssd"  # [ pd-ssd, pd-standard]
DISK_SIZE = 200  # GB

disk_spec = {"boot_disk_type": DISK_TYPE, "boot_disk_size_gb": DISK_SIZE}

### 准备您的容器规范

现在为您的自定义训练容器定义容器规范：

- `image_uri`：自定义容器镜像。
- `args`：要传递给作为容器入口点设置的可执行文件的命令行参数。
  - `--model-dir`：在我们的演示中，我们使用此命令行参数来指定存储模型文件的位置。
      - 直接：您将云存储位置作为命令行参数传递给您的训练脚本（设置变量`DIRECT = True`），或者
      - 间接：服务将云存储位置作为环境变量`AIP_MODEL_DIR`传递给您的训练脚本（设置变量`DIRECT = False`）。在这种情况下，您在作业规范中告诉服务模型文件位置。
  - `"--epochs=" + EPOCHS`：训练的时代数。
  - `"--steps=" + STEPS`：每个时代的步数。

In [ ]:
JOB_NAME = "_custom_container" + TIMESTAMP
MODEL_DIR = "{}/{}".format(BUCKET_NAME, JOB_NAME)

EPOCHS = 20
STEPS = 100

DIRECT = True
if DIRECT:
    CMDARGS = [
        "--model-dir=" + MODEL_DIR,
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]
else:
    CMDARGS = [
        "--epochs=" + str(EPOCHS),
        "--steps=" + str(STEPS),
    ]

container_spec = {
    "image_uri": TRAIN_IMAGE,
    "args": CMDARGS,
}

### 定义工作人员池规范

接下来，您需要为自定义训练任务定义工作人员池规范。工作人员池规范将包括以下内容：

- `replica_count`：要创建的该机型实例的数量。
- `machine_spec`：硬件规范。
- `disk_spec`：（可选）磁盘存储规范。
- `container_spec`：要安装在VM实例上的Docker容器。

In [ ]:
worker_pool_spec = [
    {
        "replica_count": 1,
        "machine_spec": machine_spec,
        "container_spec": container_spec,
        "disk_spec": disk_spec,
    }
]

### 组装工作规范

现在组装自定义工作规范的完整描述：

- `display_name`: 您为此自定义工作指定的可读名称。
- `job_spec`: 自定义工作的规范。
    - `worker_pool_specs`: 机器 VM 实例的规范。
    - `base_output_directory`: 这告诉服务在哪个 Cloud Storage 位置保存模型工件（当变量`DIRECT = False`时）。然后，服务将将位置传递给训练脚本作为环境变量`AIP_MODEL_DIR`，路径将为形式：

                <output_uri_prefix>/model

In [ ]:
if DIRECT:
    job_spec = {"worker_pool_specs": worker_pool_spec}
else:
    job_spec = {
        "worker_pool_specs": worker_pool_spec,
        "base_output_directory": {"output_uri_prefix": MODEL_DIR},
    }

custom_job = {"display_name": JOB_NAME, "job_spec": job_spec}

### 训练模型

现在在Vertex上开始对您的自定义训练作业进行训练。使用这个帮助函数 `create_custom_job`，它接受以下参数：

-`custom_job`: 自定义作业的规范。

帮助函数调用作业客户端服务的 `create_custom_job` 方法，具有以下参数：

-`parent`: 指向`Dataset`、`Model`和`Endpoint`资源的Vertex位置路径。
-`custom_job`: 自定义作业的规范。

您将在`response`对象中显示返回的一些字段，其中最感兴趣的两个是：

`response.name`: 分配给此自定义训练作业的Vertex完全限定标识符。您保存此标识符以在后续步骤中使用。

`response.state`: 自定义训练作业的当前状态。

In [ ]:
def create_custom_job(custom_job):
    response = clients["job"].create_custom_job(parent=PARENT, custom_job=custom_job)
    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = create_custom_job(custom_job)

现在获取您创建的自定义作业的唯一标识符。

In [ ]:
# The full unique ID for the custom job
job_id = response.name
# The short numeric ID for the custom job
job_short_id = job_id.split("/")[-1]

print(job_id)

获取有关自定义作业的信息

接下来，使用这个辅助函数`get_custom_job`，它接受以下参数：

- `name`：自定义作业的 Vertex 完全限定符标识符。

该辅助函数调用作业客户端服务的`get_custom_job`方法，带有以下参数：

- `name`：自定义作业的 Vertex 完全限定符标识符。

如果您还记得，您在调用`create_custom_job`方法时在`response.name`字段中获得了自定义作业的 Vertex 完全限定符标识符，并将该标识符保存在变量`job_id`中。

In [ ]:
def get_custom_job(name, silent=False):
    response = clients["job"].get_custom_job(name=name)
    if silent:
        return response

    print("name:", response.name)
    print("display_name:", response.display_name)
    print("state:", response.state)
    print("create_time:", response.create_time)
    print("update_time:", response.update_time)
    return response


response = get_custom_job(job_id)

# 部署

训练以上模型可能需要长达20分钟的时间。

一旦您的模型训练完成，您可以通过将“end_time”减去“start_time”来计算训练模型所花费的实际时间。对于您的模型，我们需要知道保存模型的位置，该Python脚本保存在您本地云存储桶中的“MODEL_DIR + '/saved_model.pb'”。

In [ ]:
while True:
    response = get_custom_job(job_id, True)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        model_path_to_deploy = None
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        if not DIRECT:
            MODEL_DIR = MODEL_DIR + "/model"
        model_path_to_deploy = MODEL_DIR
        print("Training Time:", response.update_time - response.create_time)
        break
    time.sleep(60)

print("model_to_deploy:", model_path_to_deploy)

加载保存的模型

您的模型以 TensorFlow SavedModel 格式存储在 Cloud Storage 存储桶中。现在从 Cloud Storage 存储桶中加载它，然后您可以执行一些操作，如评估模型和进行预测。

要加载，您可以使用 TF.Keras 的 `model.load_model()` 方法，将保存模型的 Cloud Storage 路径传递给它 -- 由 `MODEL_DIR` 指定。

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(MODEL_DIR)

## 评估模型

现在让我们看看模型有多好。

### 加载评估数据

您将从 `tf.keras.datasets` 加载波士顿房屋测试（留出）数据，使用方法 `load_data()`。这将返回数据集作为两个元素的元组。第一个元素是训练数据，第二个是测试数据。每个元素也是包含两个元素的元组：特征数据和相应的标签（业主占有住宅的中位数价值）。

您不需要训练数据，这就是为什么我们将其加载为 `(_, _)`。

在可以对数据进行评估之前，您需要对其进行预处理：

x_test:
1. 将每列的数据归一化（重新缩放），即通过每个值除以该列的最大值。这将用介于 0 和 1 之间的 32 位浮点数替换每个单个值。

In [ ]:
import numpy as np
from tensorflow.keras.datasets import boston_housing

(_, _), (x_test, y_test) = boston_housing.load_data(
    path="boston_housing.npz", test_split=0.2, seed=113
)


def scale(feature):
    max = np.max(feature)
    feature = (feature / max).astype(np.float32)
    return feature


# Let's save one data item that has not been scaled
x_test_notscaled = x_test[0:1].copy()

for _ in range(13):
    x_test[_] = scale(x_test[_])
x_test = x_test.astype(np.float32)

print(x_test.shape, x_test.dtype, y_test.shape)
print("scaled", x_test[0])
print("unscaled", x_test_notscaled)

### 执行模型评估

现在评估一下自定义工作中的模型表现如何。

In [ ]:
model.evaluate(x_test, y_test)

上传模型以进行服务

接下来，您将从自定义作业上传您的 TF.Keras 模型到 Vertex `Model` 服务，这将为您的自定义模型创建一个 Vertex `Model` 资源。在上传过程中，您需要定义一个服务函数来将数据转换为模型所期望的格式。如果您将编码数据发送到 Vertex，则您的服务函数将确保在将数据作为输入传递给模型之前，在模型服务器上对数据进行解码。

### 服务函数的工作原理

当您向在线预测服务器发送请求时，请求将被 HTTP 服务器接收。HTTP 服务器从 HTTP 请求内容体中提取预测请求。提取的预测请求将被转发到服务函数。对于Google预构建的预测容器，请求内容将作为`tf.string`传递给服务函数。

服务函数由两部分组成：

- `预处理函数`：
  - 将输入(`tf.string`)转换为基础模型(动态图)的输入形状和数据类型。
  - 执行与训练基础模型期间相同的数据预处理 -- 例如，标准化，缩放等。
- `后处理函数`：
  - 将模型输出转换为接收应用程序所期望的格式 -- 例如，压缩输出。
  - 为接收应用程序打包输出 -- 例如，添加标题，制作 JSON 对象等。

预处理和后处理功能转换为与模型融合的静态图。基础模型的输出传递给后处理函数。后处理函数将转换/打包后的输出传递回HTTP服务器。HTTP服务器将输出作为HTTP响应内容返回。

在构建用于TF.Keras模型的服务函数时，您需要考虑的一点是它们作为静态图运行。这意味着您不能使用需要动态图的TF图操作。如果这样做，您将在服务函数的编译过程中遇到错误，该错误将指示您使用了不受支持的EagerTensor。

获取serving函数签名

您可以通过重新加载模型到内存，并查询每个层对应的签名来获取模型的输入和输出层的签名。

在进行预测请求时，您需要将请求路由到serving函数而不是模型，因此您需要知道serving函数的输入层名称--当您进行预测请求时将会使用。

In [ ]:
loaded = tf.saved_model.load(model_path_to_deploy)

serving_input = list(
    loaded.signatures["serving_default"].structured_input_signature[1].keys()
)[0]
print("Serving function input:", serving_input)

上传模型

使用这个辅助函数`upload_model`来上传你的模型，存储在SavedModel格式中，到`Model`服务，该服务将为你的模型实例化一个Vertex`Model`资源实例。一旦你完成了这个步骤，你可以像其他Vertex`Model`资源实例一样使用`Model`资源实例，比如部署到用于提供预测的`Endpoint`资源。

这个辅助函数接受以下参数：

- `display_name`：对`Endpoint`服务的可读名称。
- `image_uri`：用于模型部署的容器镜像。
- `model_uri`：我们的SavedModel艺术品的云存储路径。对于本教程，这是`trainer/task.py`保存模型艺术品的Cloud Storage位置，我们在变量`MODEL_DIR`中指定了。

辅助函数调用`Model`客户端服务的`upload_model`方法，该方法接受以下参数：

- `parent`：为`Dataset`、`Model`和`Endpoint`资源指定的Vertex位置根路径。
- `model`：Vertex`Model`资源实例的规范。

现在让我们更深入地了解Vertex模型规范`model`。这是一个包含以下字段的字典对象：

- `display_name`：对`Model`资源的可读名称。
- `metadata_schema_uri`：由于您的模型是在没有Vertex`Dataset`资源的情况下构建的，您可以将此设置为空（`''`）。
- `artificat_uri`：以SavedModel格式存储模型的Cloud Storage路径。
- `container_spec`：这是将安装在`Endpoint`资源上的Docker容器的规范，从中`Model`资源将提供预测。使用之前设置的变量`DEPLOY_GPU != None`来使用GPU；否则只分配CPU。

将模型上传到Vertex Model资源会返回一个长时间运行的操作，因为可能需要一些时间。你调用response.result()，这是一个同步调用，当Vertex Model资源准备就绪时会返回。

辅助函数返回对应的Vertex Model实例的Vertex完全合格标识符upload_model_response.model。你将保存该标识符以供后续步骤使用在变量model_to_deploy_id中。

In [ ]:
IMAGE_URI = DEPLOY_IMAGE


def upload_model(display_name, image_uri, model_uri):
    model = {
        "display_name": display_name,
        "metadata_schema_uri": "",
        "artifact_uri": model_uri,
        "container_spec": {
            "image_uri": image_uri,
            "command": [],
            "args": [],
            "env": [{"name": "env_name", "value": "env_value"}],
            "ports": [{"container_port": 8080}],
            "predict_route": "",
            "health_route": "",
        },
    }
    response = clients["model"].upload_model(parent=PARENT, model=model)
    print("Long running operation:", response.operation.name)
    upload_model_response = response.result(timeout=180)
    print("upload_model_response")
    print(" model:", upload_model_response.model)
    return upload_model_response.model


model_to_deploy_id = upload_model(
    "boston-" + TIMESTAMP, IMAGE_URI, model_path_to_deploy
)

获取`Model`资源信息

现在让我们获取仅您的模型的模型信息。使用这个辅助函数 `get_model`，带有以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

这个辅助函数调用了Vertex `Model`客户端服务的方法 `get_model`，带有以下参数：

- `name`：`Model`资源的Vertex唯一标识符。

In [ ]:
def get_model(name):
    response = clients["model"].get_model(name=name)
    print(response)


get_model(model_to_deploy_id)

部署`Model`资源

现在部署经过训练的Vertex自定义`Model`资源。这需要两个步骤：

1. 为将`Model`资源部署到其中创建一个`Endpoint`资源。

2. 将`Model`资源部署到`Endpoint`资源。

### 创建一个 `Endpoint` 资源

使用这个辅助函数 `create_endpoint` 来创建一个用于部署模型以提供预测的端点，需要传入以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。

这个辅助函数使用端点客户端服务的 `create_endpoint` 方法，该方法接受以下参数：

- `display_name`：`Endpoint` 资源的人类可读名称。

创建一个 `Endpoint` 资源将返回一个长时间运行的操作，因为可能需要一些时间来为提供服务而预留 `Endpoint` 资源。您可以调用 `response.result()`，这是一个同步调用，在 `Endpoint` 资源准备就绪时将返回。辅助函数将返回 `Endpoint` 资源的 Vertex 完全合格标识符：`response.name`。

In [ ]:
ENDPOINT_NAME = "boston_endpoint-" + TIMESTAMP


def create_endpoint(display_name):
    endpoint = {"display_name": display_name}
    response = clients["endpoint"].create_endpoint(parent=PARENT, endpoint=endpoint)
    print("Long running operation:", response.operation.name)

    result = response.result(timeout=300)
    print("result")
    print(" name:", result.name)
    print(" display_name:", result.display_name)
    print(" description:", result.description)
    print(" labels:", result.labels)
    print(" create_time:", result.create_time)
    print(" update_time:", result.update_time)
    return result


result = create_endpoint(ENDPOINT_NAME)

现在获得您创建的`终端点`资源的唯一标识符。

In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### 计算实例缩放

在处理您的在线预测请求时，您有几种选项可供选择缩放计算实例：

- 单个实例：在线预测请求在单个计算实例上进行处理。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为一。

- 手动缩放：在线预测请求分配到您手动指定的固定数量的计算实例中。
  - 将计算实例的最小（`MIN_NODES`）和最大（`MAX_NODES`）数量设置为相同的节点数。当模型首次部署到实例时，固定数量的计算实例将被提供，并且在线预测请求将均匀分布在它们之间。

- 自动缩放：在线预测请求分配到可缩放数量的计算实例中。
  - 将计算实例的最小（`MIN_NODES`）数量设置为首次部署模型时提供和取消部署的实例数量，并将最大（`MAX_NODES`）数量设置为根据负载情况来提供的实例数量。

计算实例的最小数量对应于字段`min_replica_count`，而最大计算实例数量对应于字段`max_replica_count`，在您的后续部署请求中。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

### 将 `Model` 资源部署到 `Endpoint` 资源

使用这个辅助函数 `deploy_model` 将 `Model` 资源部署到您为提供预测创建的 `Endpoint` 资源，使用以下参数：

- `model`：要从训练管道上传（部署）的 Vertex 完全限定模型标识符。
- `deploy_model_display_name`：部署模型的可读名称。
- `endpoint`：要部署模型的 Vertex 完全限定终端点标识符。

辅助函数调用 `Endpoint` 客户端服务的 `deploy_model` 方法，该方法采用以下参数：

- `endpoint`：要将 `Model` 资源部署到的 Vertex 完全限定 `Endpoint` 资源标识符。
- `deployed_model`：部署模型的需求规范。
- `traffic_split`：流向此模型的终端点流量百分比，指定为一个或多个键/值对的字典。
   - 如果只有一个模型，则指定为 **{ "0": 100 }**，其中 "0" 指代要上传的这个模型，而 100 表示所有流量的 100%。
   - 如果终端点上已经存在要分配流量的模型，则使用 `model_id` 来指定为 **{ "0": 百分比, model_id: 百分比, ... }**，其中 `model_id` 是要部署到终端点的现有模型的模型 id。各百分比应相加为 100。

现在让我们更深入地探讨 `deployed_model` 参数。该参数被指定为一个带有最小所需字段的 Python 字典：

- `model`：要部署的 Vertex 完全限定模型标识符。
- `display_name`：部署模型的可读名称。
- `disable_container_logging`：禁用容器事件的日志记录，例如执行失败（默认情况下启用容器日志记录）。通常在部署调试时启用容器日志记录，然后在方便生产时禁用。
- `dedicated_resources`：指定为为提供预测请求而扩展的计算实例（复制品）的数量。
  - `machine_spec`：要配置的计算实例。 使用您之前设置的变量 `DEPLOY_GPU != None` 来使用 GPU；否则只分配 CPU。
  - `min_replica_count`：最初提供的计算实例数量，您之前设置为 `MIN_NODES` 变量。
  - `max_replica_count`：可扩展到的最大计算实例数量，您之前设置为 `MAX_NODES` 变量。

#### 流量分配

现在让我们更深入地探讨 `traffic_split` 参数。该参数被指定为一个 Python 字典。起初可能有点令人困惑。让我解释一下，您可以将模型的多个实例部署到一个终端点，然后设定每个实例接收多少流量（百分比）。

为什么要这样做？也许您已经在生产环境中部署了以前的版本--我们称之为 v1。您在 v2 上获得了更好的模型评估，但在将其部署到生产环境之前您并不确定它是否真的更好。因此，在流量分配的情况下，您可能希望将 v2 部署到与 v1 相同的终端点，但只占用 10% 的流量。这样，您可以在不干扰大多数用户的情况下监视其表现，直到做出最终决定。

#### 响应

该方法返回一个长时间运行的操作 `response`。我们将同步等待操作完成，通过调用 `response.result()`，它会阻塞直到模型部署完成。如果这是第一次将模型部署到终端点，可能需要额外几分钟来完成资源的提供。

In [ ]:
DEPLOYED_NAME = "boston_deployed-" + TIMESTAMP


def deploy_model(
    model, deployed_model_display_name, endpoint, traffic_split={"0": 100}
):

    if DEPLOY_GPU:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_type": DEPLOY_GPU,
            "accelerator_count": DEPLOY_NGPU,
        }
    else:
        machine_spec = {
            "machine_type": DEPLOY_COMPUTE,
            "accelerator_count": 0,
        }

    deployed_model = {
        "model": model,
        "display_name": deployed_model_display_name,
        "dedicated_resources": {
            "min_replica_count": MIN_NODES,
            "max_replica_count": MAX_NODES,
            "machine_spec": machine_spec,
        },
        "disable_container_logging": False,
    }

    response = clients["endpoint"].deploy_model(
        endpoint=endpoint, deployed_model=deployed_model, traffic_split=traffic_split
    )

    print("Long running operation:", response.operation.name)
    result = response.result()
    print("result")
    deployed_model = result.deployed_model
    print(" deployed_model")
    print("  id:", deployed_model.id)
    print("  model:", deployed_model.model)
    print("  display_name:", deployed_model.display_name)
    print("  create_time:", deployed_model.create_time)

    return deployed_model.id


deployed_model_id = deploy_model(model_to_deploy_id, DEPLOYED_NAME, endpoint_id)

进行在线预测请求

现在对您部署的模型进行在线预测。

获取测试项目

你将从数据集的测试（留置）部分中选择一个示例作为测试项目。

In [ ]:
test_item = x_test[0]
test_label = y_test[0]
print(test_item.shape)

### 发送预测请求

好的，现在您有一个测试数据项。使用这个辅助函数 `predict_data`，它接受以下参数：

- `data`：测试数据项，作为一个浮点数值的 numpy 1D 数组。
- `endpoint`：Vertex 的完全限定标识符，指定 `Endpoint` 资源所在的位置，`Model` 资源已部署在其中。
- `parameters_dict`：用于服务的其他参数。

该函数使用预测客户端服务，并调用 `predict` 方法，使用以下参数：

- `endpoint`：Vertex 的完全限定标识符，指定 `Model` 资源部署在其中的 `Endpoint` 资源的位置。
- `instances`：要进行预测的实例（数据项）列表。
- `parameters`：用于服务的其他参数。

为了将测试数据传递给预测服务，您需要将其打包成可传输到服务二进制文件的形式，具体步骤如下：

    1. 将数据项从 1D numpy 数组转换为 1D Python 列表。
    2. 将预测请求转换为序列化的 Google protobuf（通过 `json_format.ParseDict()`）

预测请求中的每个实例都是一个字典条目，形式如下：

                        {input_name: content}

- `input_name`：底层模型的输入层的名称。
- `content`：数据项，作为一个 1D Python 列表。

由于 `predict()` 服务可以处理多个数据项（实例），您将单个数据项作为一个数据项列表发送。最后一步，您将实例列表转换为 Google 的 protobuf 格式，这是我们传递给 `predict()` 服务的内容。

`response` 对象返回一个列表，列表中的每个元素对应请求中的相应图像。您将在每次预测输出中看到：

- `predictions`：以 1K 美元为单位的房屋中位数预测值。

In [ ]:
def predict_data(data, endpoint, parameters_dict):
    parameters = json_format.ParseDict(parameters_dict, Value())

    # The format of each instance should conform to the deployed model's prediction input schema.
    instances_list = [{serving_input: data.tolist()}]
    instances = [json_format.ParseDict(s, Value()) for s in instances_list]

    response = clients["prediction"].predict(
        endpoint=endpoint, instances=instances, parameters=parameters
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    predictions = response.predictions
    print("predictions")
    for prediction in predictions:
        print(" prediction:", prediction)


predict_data(test_item, endpoint_id, None)

取消部署 `Model` 资源

现在取消部署您的 `Model` 资源，从提供 `Endpoint` 资源。 使用这个辅助函数 `undeploy_model`，它接受以下参数：

- `deployed_model_id`：当 `Model` 资源被部署到时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 被部署到的 Vertex 完全限定标识符为 `Endpoint` 资源。

此函数调用端点客户端服务的方法 `undeploy_model`，带有以下参数：

- `deployed_model_id`：当 `Model` 资源被部署时端点服务返回的模型部署标识符。
- `endpoint`：`Model` 资源部署到的 Vertex 完全限定标识符为 `Endpoint` 资源。
- `traffic_split`：如何在 `Endpoint` 资源上的剩余部署模型中拆分流量。

由于这是 `Endpoint` 资源上唯一部署的模型，所以您可以简单地通过将其设置为空来留下 `traffic_split`。

In [ ]:
def undeploy_model(deployed_model_id, endpoint):
    response = clients["endpoint"].undeploy_model(
        endpoint=endpoint, deployed_model_id=deployed_model_id, traffic_split={}
    )
    print(response)


undeploy_model(deployed_model_id, endpoint_id)

清理

要清理此项目中使用的所有GCP资源，您可以[删除用于教程的GCP项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在此教程中创建的各个资源：

- 数据集
- 流水线
- 模型
- 端点
- 批处理作业
- 自定义作业
- 超参数调整作业
- 云存储桶

In [ ]:
delete_dataset = True
delete_pipeline = True
delete_model = True
delete_endpoint = True
delete_batchjob = True
delete_customjob = True
delete_hptjob = True
delete_bucket = True

# Delete the dataset using the Vertex fully qualified identifier for the dataset
try:
    if delete_dataset and "dataset_id" in globals():
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex fully qualified identifier for the pipeline
try:
    if delete_pipeline and "pipeline_id" in globals():
        clients["pipeline"].delete_training_pipeline(name=pipeline_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex fully qualified identifier for the model
try:
    if delete_model and "model_to_deploy_id" in globals():
        clients["model"].delete_model(name=model_to_deploy_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex fully qualified identifier for the endpoint
try:
    if delete_endpoint and "endpoint_id" in globals():
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex fully qualified identifier for the batch job
try:
    if delete_batchjob and "batch_job_id" in globals():
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the custom job using the Vertex fully qualified identifier for the custom job
try:
    if delete_customjob and "job_id" in globals():
        clients["job"].delete_custom_job(name=job_id)
except Exception as e:
    print(e)

# Delete the hyperparameter tuning job using the Vertex fully qualified identifier for the hyperparameter tuning job
try:
    if delete_hptjob and "hpt_job_id" in globals():
        clients["job"].delete_hyperparameter_tuning_job(name=hpt_job_id)
except Exception as e:
    print(e)

if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r $BUCKET_NAME